<a href="https://colab.research.google.com/github/majauhar/DL_MVA/blob/main/SISR_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Alternative Method: Facebook Fvcore
# !git clone https://github.com/majauhar/fvcore.git
# !cd fvcore
# !pip install fvcore -q

In [2]:
# from fvcore.nn import FlopCountAnalysis

In [1]:
!git clone https://github.com/majauhar/DL_MVA.git

Cloning into 'DL_MVA'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 73 (delta 24), reused 66 (delta 19), pack-reused 0
Receiving objects: 100% (73/73), 35.54 KiB | 3.55 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
cd DL_MVA/

/content/DL_MVA


In [5]:
# import fvcore

In [6]:
# dir(fvcore)

In [7]:
# from fvcore import packages

In [8]:
# !git clone https://github.com/hellloxiaotian/LESRCNN.git

In [3]:
ls

lesrcnn/  omni/  utils/


In [4]:
from utils.efficiency_results import get_model_flops, get_model_activation

In [5]:
import os
import json
import time
import importlib
import argparse
import numpy as np
from collections import OrderedDict
import torch
import torch.nn as nn
from torch.autograd import Variable
from PIL import Image
import cv2

In [7]:
from lesrcnn.model import Net

In [9]:
model = Net()

In [10]:
input_dim = (3, 256, 256)  # set the input dimension
activations, num_conv = get_model_activation(model, input_dim)
activations = activations / 10 ** 6
print("{:>16s} : {:<.4f} [M]".format("#Activations", activations))
print("{:>16s} : {:<d}".format("#Conv2d", num_conv))

    #Activations : 173.7359 [M]
         #Conv2d : 26


In [11]:
flops = get_model_flops(model, input_dim, False)
flops = flops / 10 ** 9
print("{:>16s} : {:<.4f} [G]".format("FLOPs", flops))

           FLOPs : 80.1813 [G]


In [16]:
num_parameters = sum(map(lambda x: x.numel(), model.parameters()))
num_parameters = num_parameters / 10 ** 6
print("{:>16s} : {:<.4f} [M]".format("#Params", num_parameters))

         #Params : 0.7740 [M]


In [8]:
!pip install einops -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.8 MB/s eta 0:00:00


In [6]:
from omni.model import OmniSR

In [9]:
model = OmniSR()

window_size: 8
with_pe True
ffn_bias: 1
window_size: 8
with_pe True
ffn_bias: 1
window_size: 8
with_pe True
ffn_bias: 1
window_size: 8
with_pe True
ffn_bias: 1
window_size: 8
with_pe True
ffn_bias: 1


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
input_dim = (3, 256, 256)  # set the input dimension
activations, num_conv = get_model_activation(model, input_dim)
activations = activations / 10 ** 6
print("{:>16s} : {:<.4f} [M]".format("#Activations", activations))
print("{:>16s} : {:<d}".format("#Conv2d", num_conv))

In [7]:
# # Alternative Method: Facebook Fvcore
# !git clone https://github.com/majauhar/fvcore.git
# # !cd fvcore
# !pip install -e fvcore

Cloning into 'fvcore'...
remote: Enumerating objects: 1689, done.
remote: Counting objects: 100% (416/416), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 1689 (delta 317), reused 371 (delta 296), pack-reused 1273
Receiving objects: 100% (1689/1689), 454.81 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (1105/1105), done.
Obtaining file:///content/fvcore
  Preparing metadata (setup.py) ... done
  Running setup.py develop for fvcore


In [2]:
# !rm -r fvcore

In [13]:
# !pip uninstall fvcore -y

In [10]:
# cd sample_data

/content/sample_data


In [11]:
# import fvcore

In [12]:
# from fvcore import nn

ImportError: cannot import name 'nn' from 'fvcore' (unknown location)

In [17]:
input = torch.randn(1, 3, 256, 256)

In [19]:
flops = FlopCountAnalysis(model, input)

In [20]:
flops.total()

274665635840